In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from math import exp
from tqdm.notebook import tqdm
from datetime import datetime

In [2]:
from Quordle import *
from get_word_funcs import *
from word_functions import *
from Environment import *
#from AdvantageActorCritic import *
env = Environment()

In [3]:
def test_state(env):
    state = env.state
    answer = env.quordle.answer
    guesses = env.quordle.guesses
    hints = env.quordle.hints
    first_27 = state[:27]
    for it in range(4):
        print('Next Answer: ' + str(answer[it]))
        next_26 = state[27 + 26*it:27+26*(it+1)]
        guess_info = state[131 + 390*it:131 + 390*(it + 1)]
        print('Answer:' )
        print(answer)
        print('Guesses: ')
        print(guesses)
        print('Hints: ')
        print(hints)
        print('Number of Guesses Remaining: '+str(first_27[0]))
        letters_used = []
        in_word = []
        for i in range(1, 27):
            if first_27[i] == 1:
                char = chr(i-1 + ord('a'))
                letters_used.append(char)
            if next_26[i-1] == 1:
                char = chr(i-1 + ord('a'))
                in_word.append(char)
        print('Letters Used:')
        print(letters_used)
        print('Letters in Word: ')
        print(in_word)
        for position in range(5):
            definite_letters = []
            possible_letters = []
            no_letters = []
            listlist = [no_letters, possible_letters, definite_letters]
            for letter in range(26):
                char = chr(letter + ord('a'))
                index = position*26*3 + letter*3
                sublist = guess_info[index : index + 3]
                for j in range(3):
                    if sublist[j] > 0:
                        listlist[j].append(char)
            print('Position: '+str(position))
            print('No: ')
            print(no_letters)
            print('Maybe: ')
            print(possible_letters)
            print('Yes: ')
            print(definite_letters)

In [4]:
#p = Player()
env = Environment()
ind = 0
while not env.quordle.over:
    env.step(ind)
    test_state(env)
    ind += 1

Next Answer: smelt
Answer:
['smelt' 'topic' 'dally' 'scowl']
Guesses: 
['cigar']
Hints: 
[[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0]]]
Number of Guesses Remaining: 8.0
Letters Used:
['a', 'c', 'g', 'i', 'r']
Letters in Word: 
[]
Position: 0
No: 
['a', 'c', 'g', 'i', 'r']
Maybe: 
['b', 'd', 'e', 'f', 'h', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Yes: 
[]
Position: 1
No: 
['a', 'c', 'g', 'i', 'r']
Maybe: 
['b', 'd', 'e', 'f', 'h', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Yes: 
[]
Position: 2
No: 
['a', 'c', 'g', 'i', 'r']
Maybe: 
['b', 'd', 'e', 'f', 'h', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Yes: 
[]
Position: 3
No: 
['a', 'c', 'g', 'i', 'r']
Maybe: 
['b', 'd', 'e', 'f', 'h', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Yes: 
[]
Position: 4
No: 
['a', 'c', 'g', 'i', 'r']

In [7]:
env.state

array([-3.,  1.,  1., ...,  0.,  1.,  0.])

In [5]:
critic_loss_weight = 0.5
aator_loss_weight = 1
entropy_loss_weight = 0.05


class AdvantageActorCritic(keras.Model):
    def __init__(self, mx, actions_list):
        super().__init__()
        # self.input = [417]
        self.mx = mx
        self.init = keras.initializers.random_uniform()
        self.actions_list = actions_list
        self.n_outputs = 130
        self.leaky = layers.LeakyReLU()
        self.dense1 = layers.Dense(256, activation=self.leaky, kernel_initializer=self.init)
        self.dense2 = layers.Dense(256, activation=self.leaky, kernel_initializer=self.init)
        self.dense3 = layers.Dense(256, activation=self.leaky, kernel_initializer=self.init)
        self.dense4 = layers.Dense(256, activation=self.leaky, kernel_initializer=self.init)
        self.policy1 = layers.Dense(self.n_outputs, kernel_initializer=self.init)
        self.value = layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        policy = self.policy1(x)
        policy = tf.linalg.matmul(self.mx, policy, transpose_b=True)
        policy_logits = tf.linalg.matrix_transpose(policy)
        value = self.value(x)
        return value, policy_logits

    def action_value(self, state):
        value, logits = self.call(state)
        action = tf.random.categorical(logits, 1)
        return action, value


def critic_loss(rewards, predicted_values):
    return keras.losses.mean_squared_error(rewards, predicted_values) * critic_loss_weight


def actor_loss(combined, policy_logits):
    actions = combined[:, 0]
    advantages = combined[:, 1]
    sparse_ce = keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                           reduction=tf.keras.losses.Reduction.SUM)

    actions = tf.cast(actions, tf.int32)
    policy_loss = sparse_ce(actions, policy_logits, sample_weight=advantages)

    probs = tf.nn.softmax(policy_logits)
    entropy_loss = keras.losses.categorical_crossentropy(probs, probs)

    return policy_loss * actor_loss_weight - entropy_loss * entropy_loss_weight

def compute_advantages(rewards, values, dones):
    discount_factor = 0.7
    advantages = np.zeros(len(rewards))
    advantages[-1] = rewards[-1] - values[-1]
    out_rewards = np.zeros(len(rewards))
    out_rewards[-1] = rewards[-1]
    n = len(rewards)
    for i in reversed(range(len(rewards)-1)):
        out_rewards[i] = rewards[i] + (1 - dones[i])*discount_factor*out_rewards[i+1]

    advantages = out_rewards - values
    return out_rewards, advantages



In [6]:
def test_set(word_list, indices, print_guesses = False):
    test_rewards = []
    env = Environment()
    wins = 0
    pick = np.random.choice(range(len(word_list)))
    for i in range(len(word_list)):
        answer = word_list[i]
        rewards_in_game = 0
        env.reset(answer)
        while not env.wordle.over:
            _, action_probs = AAC(env.state[np.newaxis])
            action_probs = tf.nn.softmax(action_probs).numpy()[0]
            action = np.random.choice(indices, 1, p=action_probs)[0]
            s, reward, o = env.step(action)
            rewards_in_game += reward
        test_rewards.append(rewards_in_game)
        wins += env.wordle.win
        if i == pick:
            print(env.wordle.answer)
            print(env.wordle.guesses)
        if print_guesses:
            print(env.wordle.answer)
            print(env.wordle.guesses)
    print('Wins: '+str(wins))
    return wins

In [7]:
freqs = np.load('wordfrequencylist.npy')
top1000ind = np.argsort(freqs,axis = 0)[-1000:]
top1000ind = top1000ind.flatten()
top1000ind = np.sort(top1000ind)

In [8]:
total_words = get_words()
one_hot = one_hot_words().astype(np.float32)
one_hot_1000 = one_hot[top1000ind, :]
top1000words = total_words[top1000ind]

In [9]:
AAC = AdvantageActorCritic(one_hot_1000, top1000ind)
AAC.load_weights('EMH_Model_Weights/H_AAC_weights_519')
AAC.compile(optimizer=keras.optimizers.Adam(), loss = [critic_loss, actor_loss])

In [10]:
num_games = 2
#critic_loss = []
#actor_loss = []
answers = top1000words
##Actor-Critic Attempt, Train every n games
memory = Memory()
memory.clear()
#num_games_per_train = batch_size
env = Environment(np.random.choice(answers))
for game in tqdm(range(1, num_games+1)):
    total_training_rewards = 0
    env.reset(np.random.choice(answers, 1)[0])
    state = env.state
    while not env.wordle.over:
        state = tf.cast(env.state, tf.uint16)
        action_ind, value = AAC.action_value(state[np.newaxis])
        action_ind = action_ind.numpy()[0][0]
        action = top1000ind[action_ind]
        next_state, reward, done = env.step(action)

        memory.add(state, action_ind, value.numpy()[0][0], done, reward)
        total_training_rewards += reward
    print(env.wordle.answer)
    print(env.wordle.guesses)
    print(env.wordle.hints)

    states = memory.states
    actions = memory.actions
    values = memory.values
    dones = memory.dones
    rewards = memory.rewards


rewards, advantages = compute_advantages(rewards, values, dones)
    #combined = np.zeros((len(actions), 2))
    #combined[:, 0] = actions
    #combined[:, 1] = advantages
    #rewards = np.resize(rewards, (len(rewards), 1))
    #loss = AAC.train_on_batch(x = tf.stack(states), y = (rewards, combined))
    #critic_loss.append(loss[1])
    #actor_loss.append(loss[2])
        #memory.clear()


  0%|          | 0/2 [00:00<?, ?it/s]

sluts
['tears', 'suits', 'south', 'sucks', 'sucks', 'sucks']
[[1.0, 0.0, 0.0, 0.0, 2.0], [2.0, 1.0, 0.0, 2.0, 2.0], [2.0, 0.0, 2.0, 2.0, 0.0], [2.0, 1.0, 0.0, 0.0, 2.0], [2.0, 1.0, 0.0, 0.0, 2.0], [2.0, 1.0, 0.0, 0.0, 2.0]]
wrote
['tears', 'wrote']
[[1.0, 1.0, 0.0, 1.0, 0.0], [2.0, 2.0, 2.0, 2.0, 2.0]]


In [11]:
memory.rewards

[0, 0, 0, 0, -2, -12, 0, 10]

In [12]:
rewards

array([ -2.49704,  -3.5672 ,  -5.096  ,  -7.28   , -10.4    , -12.     ,
         7.     ,  10.     ])

In [13]:
advantages

array([-18.15472623, -23.10776931, -25.46452264, -25.47668007,
       -27.26886406, -28.78431511,  -8.65768623,  -6.93361282])

In [14]:
memory.dones

[False, False, False, False, False, True, False, True]